In [1]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from scipy import stats
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy

import Config
from Config import *
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC


# Generate data

In [2]:
def randgen_DNA(n, query = "", c_len = 4, nMis = 0, debug = False):
    
    if debug:
        print("Generating rand seq of length", n)
        
    #DNA = "ACGT"
    
    substring = str(query)
    if nMis != 0 and len(substring) >= nMis:
        idxs = np.random.choice(len(substring), size = nMis, replace=False)
        lvls = np.random.randint(1, 4, nMis)
        for i in range(len(idxs)):
            substring = substring[:idxs[i]] + str( (int(substring[idxs[i]]) + lvls[i]) % 4) + substring[idxs[i]+1:]
    substring = str(substring)
    
    m = n - len(substring)
    if m == 0:
        return substring
    datum = [str(np.random.randint(0,4)) for _ in range(m)]
    place = 0
    if c_len is None:
        place = np.random.randint(0,m+1)
    else:
        place = np.random.randint(0,m+1)//c_len * c_len
    string = "".join(datum)
    string = string[:place] + substring + string[place:]
    assert len(string) == n
    assert string.find(substring) != -1
    return string, place


In [3]:
def datagen(n_len, c_len, nMi, debug = False):
    
    q_len = 2**8
    q_amt = 100
    n_amt = 100

    #n_len = 2**8 + 2 ** 10
    #c_len = 4

    queries = np.asarray([randgen_DNA(q_len) for _ in range(q_amt)])
    queries = queries[:, 0]
    #print(queries[0])

    data = np.asarray([randgen_DNA(n_len, queries[i], c_len, nMis) for i in range(n_amt)])
    refs = data[:,0]
    labs = data[:,1]
    
    q_file = open("query.txt", "w")
    for q in queries:
        q_file.write(q+"\n")
    q_file.close()
    
    r_file = open("ref.txt", "w")
    for r in refs:
        r_file.write(r+"\n")
    r_file.close()
    
    l_file = open("lab.txt", "w")
    for l in labs:
        l_file.write(l+"\n")
    l_file.close()

# Test accuracy

In [4]:
def perm_inv(p):
    s = np.empty_like(p)
    s[p] = np.arange(p.size)
    return s
def perm_exp(p, n):
    s = np.arange(len(p))
    for i in range(n):
        s = s[p]
    return s
def preprocess_bind(basis, c_len, perm):
    mapping = dict()
    if c_len == 0:
        mapping[""] = -np.ones(len(basis[0]))
        return mapping
    suf_mapping = preprocess_bind(basis, c_len - 1, perm)
    for suffix in suf_mapping.keys():
        suf_enc = suf_mapping[suffix]
        perm_enc = suf_enc[perm]
        for i in range(4):
            currkey = str(i) + suffix
            mapping[currkey] = - (basis[i] * perm_enc)
    return mapping
def bind(chunk, basis, perm = None):
    D = len(basis[0])
    if perm is None:
        perm = np.arange(D)
    result = -np.ones(D)
    for c in reversed(chunk):
        result = -(result[perm] * basis[int(c)])
    return result
def bundling(vecs, perm = None):
    D = len(vecs[0])
    if perm is None:
        perm = np.arange(D)
    result = np.zeros(D)
    for vec in reversed(vecs):
        result = result[perm] + vec
    return result
def binarize(X):
    return np.where(X >= 0, 1, -1)
def chunk_string(string, c_len, overlap = False):
    if not overlap:
        return [string[c_len * i: c_len * (i+1)] for i in range(len(string)//c_len)]
    else:
        return [string[i:i + c_len] for i in range(len(string) - c_len + 1)]

# String: string of arbitrary length for encoding
# c_len: chunk length to bind together
# Note: non-overlapping by default
# s_len: substring length 
# perm1: permutation used for binding, not needed given a complete mapping 
# perm2: perm used for bundling among chunks
# perm3: perm used for bunding among substrings
def encode_DNA(mapping, string, c_len, s_len, perm2, basis = None, perm1 = None, overlap = False, debug = False):
    
    global D
    
    chunks = chunk_string(string, c_len, overlap)
    cps = s_len // c_len # Chunk per substring
    enc_strs = []
    enc_chunks = []
    
    if not overlap: 
        
        n_str =  len(chunks) - cps + 1

        # Prep prefix. Head chunk is the chunk to delete next. Initialized to 0
        if mapping is None:
            enc_chunks = [bind(chunks[i], basis, perm1) for i in range(cps - 1)]
            #print(enc_chunks)
        else:
            enc_chunks = [mapping[chunks[i]] for i in range(cps - 1)]
            #print(enc_chunks)
        #print(enc_chunks)

        head_enc_chunk = np.zeros(D)
        if len(enc_chunks) != 0:
            prefix = bundling(enc_chunks, perm2)
            prefix = prefix[perm2]
        else:
            prefix = np.zeros(D)

        perm2_inv = perm_inv(perm2)
        perm2_fin = perm_exp(perm2, cps - 1)

        for i in range(n_str):
            prefix -= head_enc_chunk
            prefix = prefix[perm2_inv]

            if mapping is None:
                curr_enc_chunk = bind(chunks[cps + i - 1], basis, perm1)
            else:
                curr_enc_chunk = mapping[chunks[cps + i - 1]] # Find the next final chunk

            curr_enc_chunk = curr_enc_chunk[perm2_fin]
            prefix += curr_enc_chunk

            if debug:
                print("Current encoded substring:", prefix)
            enc_strs.append(np.copy(prefix))

            if mapping is None:
                head_enc_chunk = bind(chunks[i], basis, perm1)
            else:
                head_enc_chunk = mapping[chunks[i]] # Retrieve the next first chunk

        enc_string = bundling(enc_strs)
        return enc_string
    else:
        pass
    
def encode_data(mapping, strings, c_len, s_len, perm2, basis = None, perm1 = None):
    encs = []
    for string in tqdm_notebook(strings, desc='epochs'):
        encs.append(encode_DNA(mapping, string, c_len, s_len, perm2,  basis, perm1))
    return encs
def evaluate(enc_q, enc_rs):
    vals = np.asarray([np.dot(enc_q, enc_r) for enc_r in enc_rs])
    order = np.asarray(list(reversed(np.argsort(vals))))
    return order, vals

In [5]:
def test(D, c_len, debug = False):
    q_file = open("query.txt", "r")
    queries = q_file.read().split("\n")
    queries = queries[:-1]
    q_file.close()

    r_file = open("ref.txt", "r")
    refs = r_file.read().split("\n")
    refs = refs[:-1]
    r_file.close()

    l_file = open("lab.txt", "r")
    labs = l_file.read().split("\n")
    labs = labs[:-1]
    l_file.close()

    if debug:
        print("Check query, ref, lable length:")
        print(len(queries))
        print(len(refs))
        print(len(labs))
    
    basis = np.random.randint(0, 2, (4,D))  * 2 - 1 
    #print(np.sum(basis, axis = 1))

    perm1 = np.random.permutation(D)
    perm2 = np.random.permutation(D)

    perm3 = np.arange(D-1)
    perm3 = np.append([D-1], perm3)

    s_len = len(queries[0])
    
    if debug:
        print("Check permutation and chunk/query length")
        print(perm1)
        print(perm2)
        print(perm3)
        print(c_len, s_len)
    
    thresh = math.log((len(queries)*len(queries[0])+len(refs)*len(refs[0]))//c_len, 4) - c_len  

    if thresh >= 0:
        # If c_len is small enough
        print("small chunk length, preprocessing chunks")
        mapping = preprocess_bind(basis, c_len, perm2)
        #print(mapping)
        enc_qs = encode_data(mapping, queries, c_len, s_len, perm_exp(perm2, c_len-1))
        enc_rs = encode_data(mapping, refs, c_len, s_len, perm_exp(perm2, c_len-1))
    else: 
        enc_qs = encode_data(None, queries, c_len, s_len, perm2, basis, perm2)
        enc_rs = encode_data(None, refs, c_len, s_len, perm2, basis, perm2)
        
        
    correct = 0
    for i in range(0, len(enc_qs)):
        print("%dth substring:################"%(i))
        order, vals = evaluate(enc_qs[i], enc_rs)

        # Handle intended matching
        if i < len(labs):
            j = np.where(order == i)[0]
            print("\t Match injected at %dth choice %d with value %d at index %s"%
                  (j+1, order[j], vals[order[j]], labs[i]))
            if j == 0 and int(labs[i]) % c_len == 0:
                correct += 1

        # Hanlde unintended perfect matching
        for j in range(len(order)):
            pos = refs[order[j]].find(queries[i])
            if pos != -1:
                print("\t Match found at %dth choice %d with value %d at index %d"%
                      (j+1, order[j], vals[order[j]], pos))

        print("\t\t Top choice %d with value %d"%(order[0], vals[order[0]]))
        print("\t\t 2nd choice %d with value %d"%(order[1], vals[order[1]])) 

    print(correct)
    return correct

# TopLevel

In [6]:
def exp(n_len, c_len, nMis, D, debug = False): 
    datagen(n_len, c_len, nMis, debug)
    test(D, c_len, debug)

In [8]:
log = dict()
D = 10000

for i in range(8):
    n_len = 2 ** 8 + 2 ** (8 + i)
    for j in range(3):
        c_len = 2 ** (2+j)
        for k in range(5):
            nMis = k
            acc = exp(n_len, c_len, nMis, D)
            log[(n_len, c_len, k)] = acc

small chunk length, preprocessing chunks


c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:114: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



0th substring:################
	 Match injected at 1th choice 0 with value 840508 at index 172
	 Match found at 1th choice 0 with value 840508 at index 172
		 Top choice 0 with value 840508
		 2nd choice 58 with value 378416
1th substring:################
	 Match injected at 1th choice 1 with value 777108 at index 12
	 Match found at 1th choice 1 with value 777108 at index 12
		 Top choice 1 with value 777108
		 2nd choice 71 with value 369900
2th substring:################
	 Match injected at 1th choice 2 with value 880900 at index 212
	 Match found at 1th choice 2 with value 880900 at index 212
		 Top choice 2 with value 880900
		 2nd choice 6 with value 407384
3th substring:################
	 Match injected at 1th choice 3 with value 792928 at index 76
	 Match found at 1th choice 3 with value 792928 at index 76
		 Top choice 3 with value 792928
		 2nd choice 86 with value 373712
4th substring:################
	 Match injected at 1th choice 4 with value 967148 at index 4
	 Match fou

small chunk length, preprocessing chunks



0th substring:################
	 Match injected at 1th choice 0 with value 690860 at index 136
		 Top choice 0 with value 690860
		 2nd choice 90 with value 316380
1th substring:################
	 Match injected at 1th choice 1 with value 815076 at index 240
		 Top choice 1 with value 815076
		 2nd choice 81 with value 349148
2th substring:################
	 Match injected at 1th choice 2 with value 944304 at index 112
		 Top choice 2 with value 944304
		 2nd choice 54 with value 456848
3th substring:################
	 Match injected at 1th choice 3 with value 833036 at index 144
		 Top choice 3 with value 833036
		 2nd choice 13 with value 338660
4th substring:################
	 Match injected at 1th choice 4 with value 864868 at index 124
		 Top choice 4 with value 864868
		 2nd choice 52 with value 368180
5th substring:################
	 Match injected at 1th choice 5 with value 843328 at index 40
		 Top choice 5 with value 843328
		 2nd choice 31 with value 339968
6th substring:##

...

		 Top choice 98 with value 193044
		 2nd choice 70 with value 182328
52th substring:################
	 Match injected at 10th choice 52 with value 88676 at index 7104
		 Top choice 11 with value 185140
		 2nd choice 9 with value 172444
53th substring:################
	 Match injected at 4th choice 53 with value 135516 at index 16688
		 Top choice 23 with value 201804
		 2nd choice 19 with value 136716
54th substring:################
	 Match injected at 9th choice 54 with value 107588 at index 528
		 Top choice 75 with value 152420
		 2nd choice 55 with value 148756
55th substring:################
	 Match injected at 15th choice 55 with value 88472 at index 22464
		 Top choice 70 with value 178664
		 2nd choice 98 with value 163276
56th substring:################
	 Match injected at 1th choice 56 with value 215420 at index 9040
		 Top choice 56 with value 215420
		 2nd choice 10 with value 137488
57th substring:################
	 Match injected at 7th choice 57 with value 107872 at ind


0th substring:################
	 Match injected at 49th choice 0 with value -1260 at index 14224
		 Top choice 70 with value 184064
		 2nd choice 72 with value 157536
1th substring:################
	 Match injected at 1th choice 1 with value 244588 at index 1632
		 Top choice 1 with value 244588
		 2nd choice 81 with value 185036
2th substring:################
	 Match injected at 1th choice 2 with value 240472 at index 11312
		 Top choice 2 with value 240472
		 2nd choice 85 with value 149112
3th substring:################
	 Match injected at 23th choice 3 with value 50876 at index 3056
		 Top choice 99 with value 164780
		 2nd choice 76 with value 149080
4th substring:################
	 Match injected at 13th choice 4 with value 107536 at index 22672
		 Top choice 99 with value 211120
		 2nd choice 11 with value 194172
5th substring:################
	 Match injected at 18th choice 5 with value 62020 at index 400
		 Top choice 99 with value 152604
		 2nd choice 94 with value 139168
6t

		 Top choice 52 with value 165252
		 2nd choice 29 with value 117732
66th substring:################
	 Match injected at 2th choice 66 with value 171336 at index 19104
		 Top choice 36 with value 208352
		 2nd choice 66 with value 171336
67th substring:################
	 Match injected at 1th choice 67 with value 259020 at index 5552
		 Top choice 67 with value 259020
		 2nd choice 4 with value 178532
68th substring:################
	 Match injected at 3th choice 68 with value 133884 at index 23696
		 Top choice 45 with value 172540
		 2nd choice 51 with value 135956
69th substring:################
	 Match injected at 1th choice 69 with value 179424 at index 11216
		 Top choice 69 with value 179424
		 2nd choice 81 with value 162992
70th substring:################
	 Match injected at 2th choice 70 with value 205344 at index 11584
		 Top choice 64 with value 211132
		 2nd choice 70 with value 205344
71th substring:################
	 Match injected at 4th choice 71 with value 122904 at 


0th substring:################
	 Match injected at 8th choice 0 with value 627836 at index 19632
		 Top choice 8 with value 771556
		 2nd choice 51 with value 685656
1th substring:################
	 Match injected at 10th choice 1 with value 611632 at index 14560
		 Top choice 70 with value 659484
		 2nd choice 21 with value 644020
2th substring:################
	 Match injected at 5th choice 2 with value 642160 at index 18672
		 Top choice 88 with value 666256
		 2nd choice 45 with value 648272
3th substring:################
	 Match injected at 27th choice 3 with value 568324 at index 11904
		 Top choice 95 with value 692484
		 2nd choice 20 with value 668952
4th substring:################
	 Match injected at 2th choice 4 with value 692812 at index 15328
		 Top choice 80 with value 708860
		 2nd choice 4 with value 692812
5th substring:################
	 Match injected at 7th choice 5 with value 631144 at index 18496
		 Top choice 60 with value 725576
		 2nd choice 88 with value 6933

	 Match injected at 4th choice 49 with value 656408 at index 12720
		 Top choice 64 with value 695652
		 2nd choice 92 with value 694012
50th substring:################
	 Match injected at 13th choice 50 with value 631976 at index 30064
		 Top choice 2 with value 726712
		 2nd choice 43 with value 715808
51th substring:################
	 Match injected at 1th choice 51 with value 752192 at index 5808
		 Top choice 51 with value 752192
		 2nd choice 85 with value 672144
52th substring:################
	 Match injected at 42th choice 52 with value 548336 at index 28688
		 Top choice 12 with value 691244
		 2nd choice 13 with value 662592
53th substring:################
	 Match injected at 1th choice 53 with value 750224 at index 2928
		 Top choice 53 with value 750224
		 2nd choice 91 with value 743744
54th substring:################
	 Match injected at 11th choice 54 with value 606708 at index 12352
		 Top choice 49 with value 675368
		 2nd choice 35 with value 673656
55th substring:###

In [9]:
for k in log.keys():
    print(k, log[k])

(512, 4, 0) None
(512, 4, 1) None
(512, 4, 2) None
(512, 4, 3) None
(512, 4, 4) None
(512, 8, 0) None
(512, 8, 1) None
(512, 8, 2) None
(512, 8, 3) None
(512, 8, 4) None
(512, 16, 0) None
(512, 16, 1) None
(512, 16, 2) None
(512, 16, 3) None
(512, 16, 4) None
(768, 4, 0) None
(768, 4, 1) None
(768, 4, 2) None
(768, 4, 3) None
(768, 4, 4) None
(768, 8, 0) None
(768, 8, 1) None
(768, 8, 2) None
(768, 8, 3) None
(768, 8, 4) None
(768, 16, 0) None
(768, 16, 1) None
(768, 16, 2) None
(768, 16, 3) None
(768, 16, 4) None
(1280, 4, 0) None
(1280, 4, 1) None
(1280, 4, 2) None
(1280, 4, 3) None
(1280, 4, 4) None
(1280, 8, 0) None
(1280, 8, 1) None
(1280, 8, 2) None
(1280, 8, 3) None
(1280, 8, 4) None
(1280, 16, 0) None
(1280, 16, 1) None
(1280, 16, 2) None
(1280, 16, 3) None
(1280, 16, 4) None
(2304, 4, 0) None
(2304, 4, 1) None
(2304, 4, 2) None
(2304, 4, 3) None
(2304, 4, 4) None
(2304, 8, 0) None
(2304, 8, 1) None
(2304, 8, 2) None
(2304, 8, 3) None
(2304, 8, 4) None
(2304, 16, 0) None
(2304,